## XGboost

在处理XGboost的输入数据的时候，我们再对于特征进行了的一步处理，对于所有的类别特征，我们首先关于每个特征都做了一个，认购率和特征内类别的分析，然后按照后续方法实现特征工程

# 实验记录：基于平滑目标编码（Target Encoding）与特征增强的 XGBoost 优化

## 1. 实验背景与问题分析
在之前的版本中，XGBoost 的准确率低于 Logistic Regression。经分析，主要原因如下：
* **编码随机性**：原先使用的 `pd.factorize` 产生的整数（0, 1, 2...）对树模型而言没有逻辑意义，无法体现职业、月份等特征与购买率的直接关联。

## 2. 核心改进方法：平滑目标编码 (Smoothed Target Encoding)
我们将类别特征替换为该类别对应的**平滑转化率**。
* **计算公式**：
    $$Encoding = \frac{(count_{category} \cdot mean_{category}) + (m \cdot mean_{global})}{count_{category} + m}$$
* **平滑系数 ($m=50$)**：当类别样本量较小时，编码值会向全局均值靠拢，有效防止过拟合。

## 3. 实验代码实现

### 3.1 定义映射学习函数
```python
def get_target_mapping(train_df, cols):
    """从训练集中学习平滑后的目标映射"""
    mapping = {}
    # 确保 target 列已转换为数值 0/1
    global_mean = train_df['subscribe'].map({'yes': 1, 'no': 0}).mean() 
    m = 50  # 平滑系数，可调参
    
    # 临时计算列用于统计
    temp_df = train_df.copy()
    temp_df['target_num'] = train_df['subscribe'].map({'yes': 1, 'no': 0})
    
    for col in cols:
        stats = temp_df.groupby(col)['target_num'].agg(['count', 'mean'])
        # 应用平滑公式
        smooth = (stats['count'] * stats['mean'] + m * global_mean) / (stats['count'] + m)
        mapping[col] = smooth.to_dict()
        
    mapping['_global_mean'] = global_mean
    return mapping

## 在此方法下，准确率可以到达96.3